In [1]:
import json
import requests

In [2]:
# load in api key
credentials = json.load(open('./apikey.json', 'r'), encoding='utf-8')

In [3]:
# GHE docs:
# curl -H "Authorization: token OAUTH-TOKEN" http(s)://[hostname]/api/v3  <-- endpoint
''' 
Declare global variables
'''
baseurl = 'git.generalassemb.ly' # of course may be different for you
header = {'Authorization': 'token {}'.format(credentials['token'])}
source_org = 'Data-science-immersive' # change this as you see fit
target_org = 'DSI-ME-1' # this one too
repo_names = []
data = { 'organization': target_org }

In [ ]:
'''
Example calls below
'''
# call the base endpoint
'''
r = requests.get('https://{}/api/v3'.format(baseurl), headers=header)
'''

# get a listing of orgs the user's api key has access to
'''
r = requests.get('https://{}/api/v3/user/orgs'.format(baseurl), headers=header)
orgs = json.loads(r.content)
for org in orgs:
    print('{}: id = {}'.format(org['login'], org['id']))
'''

### Code below will clone an entire org's repos to a local folder; use with parseme.ipynb

In [ ]:
# get a listing of all repos in the target org, 'Data-science-immersive'
# from GHE docs: GET /orgs/:org/repos

r = requests.get('https://{}/api/v3/orgs/{}/repos'.format(baseurl, source_org), \
                 headers=header)
repos = json.loads(r.content)

clones = []
for repo in repos:
    clones.append(repo['ssh_url'])
    
# show first 5
for repo in repos[:5]:
    print('{}, {}'.format(repo['name'], repo['ssh_url']))

In [ ]:
'''
This will create a place for all the cloned repos to live.
Use the parseme file, pointing to this directory, to
scan all clones repos for words
'''

%%bash
if [ ! -d "./course_dump" ]; then
    mkdir ./course_dump
    echo "created directory ./course_dump"
else
    echo "directory already exists: ./course_dump"
fi

In [ ]:
# WARNING!!! ===
# this will clone ALL repositories in the organization to ./course_dump!!
# note - has bug; will not retry cloning of repository if network fails :'(

try:
    for repo in clones:
        # made it sleepy to be a good citizen
        !cd ./course_dump && git clone $repo && sleep 1
    print("Finished cloning {} repositories into ./course_dump".format(len(clones)))
except:
    # outstanding error handling here
    print("noooooo!!")

### This will fork all repositories from a source org to a target org

In [4]:
r = requests.get('https://{}/api/v3/orgs/{}/repos'.format(baseurl, source_org), \
                 headers=header)
repos = json.loads(r.content)

fork_urls = []
for repo in repos:
    fork_urls.append(repo['forks_url'])

In [5]:
# set the target org parameter, where the forks will go
data = { 'organization': target_org }

'''
This needs to be done as a session because the GET request does the
auth, and the POST request does the forking. You CAN'T pass a cookie
between the two sessions, I tried. Couldn't find any examples of anybody
doing this on SO, etc. Was able to get it working with CURL but wanted
to use python requests lib. Trick is passing the header with BOTH the 
GET *AND* the POST requests. You can't just send the header with the 
POST request, despite what the docs lead you to believe.

School of hard knocks... damn it feels good to be a gangster
'''

with requests.Session() as s:
    # this get request authenticates the user
    r1 = s.get('https://{}/api/v3'.format(baseurl), headers=header)
    # ...and then this post request creates the fork
    for fork_url in fork_urls:
        r2 = s.post(fork_url, headers=header, data=json.dumps(data))
        print('{} successfully forked to {}'.format( \
            fork_url.split('/')[-3] + '/' + fork_url.split('/')[-2], \
            json.loads(r2.text)['full_name'])
        )

Data-science-immersive/python-101-lesson successfully forked to DSI-ME-1/templates
Data-science-immersive/python-101-lesson successfully forked to DSI-ME-1/python-101-lesson
Data-science-immersive/python-101-lesson successfully forked to DSI-ME-1/python-comprehensions-lab
Data-science-immersive/python-101-lesson successfully forked to DSI-ME-1/python-comprehensions-lesson
Data-science-immersive/python-101-lesson successfully forked to DSI-ME-1/python-functions-lesson
Data-science-immersive/python-101-lesson successfully forked to DSI-ME-1/python-intro_to_python_pokemon-lab
Data-science-immersive/python-101-lesson successfully forked to DSI-ME-1/python-iteration_control_functions-lesson
Data-science-immersive/python-101-lesson successfully forked to DSI-ME-1/statistics-central_limit_theorem-lesson
Data-science-immersive/python-101-lesson successfully forked to DSI-ME-1/statistics-continuous_distributions-lesson
Data-science-immersive/python-101-lesson successfully forked to DSI-ME-1/sta